In [1]:
import cv2
import numpy as np
from keras.models import model_from_json
import tkinter as tk
import librosa
import tensorflow as tf
from tkinter import filedialog
from tkinter import *
from tkinter import Label, Tk

In [2]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [3]:
# Load model architecture from JSON file
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [4]:
# Load model weights from H5 file
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")



Loaded model from disk


In [5]:
# load weights into new model
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
def start_detection(source):
    if source == "webcam":
        cap = cv2.VideoCapture(0)
    else:
        video_path = filedialog.askopenfilename()
        if video_path:
            cap = cv2.VideoCapture(video_path)
        else:
            return

    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            break

        frame = cv2.resize(frame, (1280, 720))
        face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
            roi_gray_frame = gray_frame[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

            emotion_prediction = emotion_model.predict(cropped_img)
            maxindex = int(np.argmax(emotion_prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        #press "s" to Quit
        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [7]:
def start_webcam_detection():
    start_detection("webcam")

def start_video_detection():
    start_detection("video")

In [8]:
def get_mel_spectrogram(wave, sample_rate):
  # Compute Mel spectrogram
  mel_spectrogram = librosa.feature.melspectrogram(y=wave, sr = sample_rate, n_fft = 2048, hop_length = 512, n_mels = 90)
  log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
  return log_mel_spectrogram

In [9]:
def noise(data):
    noise_amp = 0.035 * np.random.uniform() * np.amax(data)
    data = data + noise_amp * np.random.normal(size=data.shape[0])
    return data

In [10]:
def extract_mel_spectrogram(audio_path, trim_duration=3, show_plot=False):
    try:
        # Load audio file
        y, sr = librosa.load(audio_path, duration=trim_duration, offset=0.5)

        # Trim audio to 3 seconds
        if len(y) < sr * trim_duration:
            y = np.pad(y, (0, sr * trim_duration - len(y)))

        y = noise(y)

        # Extract mel spectrogram
        mel_spectrogram = get_mel_spectrogram(y, sr)

        # Display spectrogram
        if show_plot:
            plt.figure(figsize=(10, 4))
            librosa.display.specshow(mel_spectrogram, y_axis='mel', x_axis='time')
            plt.title('Mel Spectrogram')
            plt.colorbar(format='%+2.0f dB')
            plt.show()

        return mel_spectrogram

    except Exception as e:
        print(f"Error processing audio: {e}")
        return None



In [11]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
# Encode emotion labels
df = pd.read_csv('data_path.csv')
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(df['Emotions'])
Y[:5], Y.shape
label_encoder.classes_



array(['angry', 'calm', 'disgust', 'fear', 'happy', 'neutral', 'sad',
       'surprise'], dtype=object)

In [12]:
def detect_emotion_from_audio():
    # Load the trained audio model
    model1 = tf.keras.models.load_model('audio_model.h5')

    # Get the audio path from the GUI input
    audio_path = audio_path_entry.get()

    # Extract mel spectrogram
    mel = extract_mel_spectrogram(audio_path, trim_duration=3, show_plot=False)
    if mel is None:
        print("Error extracting mel spectrogram.")
        return

    mel = np.expand_dims(np.expand_dims(mel, axis=0), axis=-1)

    # Predict emotion
    y_pred = model1.predict(mel)
    output = np.argmax(y_pred, axis=1)
    emotion_label = label_encoder.classes_[output][0]

    # Display the predicted emotion in the GUI
    result_label.config(text=f"Predicted Emotion: {emotion_label}")

# Function to open file dialog and get audio file path
def browse_file():
    file_path = filedialog.askopenfilename()
    audio_path_entry.delete(0, 'end')
    audio_path_entry.insert(0, file_path)



In [ ]:
# Creating GUI
root = tk.Tk()
root.title("Emotion Detection")
root.geometry('800x600')
root.configure(background='#CDCDCD')

heading = Label(root,text='Emotion Detection',pady=20,font=('arial',25,'bold'))
heading.configure(background='#CDCDCD',foreground="#364156")
heading.pack()

webcam_button = tk.Button(root, text="Use Webcam for Video Detection", command=start_webcam_detection)
webcam_button.pack(pady=10)

video_button = tk.Button(root, text="Use Video File for Video Detection", command=start_video_detection)
video_button.pack(pady=10)

# press "s" to exit from webcam or video

audio_label = Label(root, text="Audio Emotion Detection", pady=10, font=('arial', 25, 'bold'))
audio_label.configure(background='#CDCDCD', foreground="#364156")
audio_label.pack()

instruction_label_audio = Label(root, text="Please enter the audio file path:")
instruction_label_audio.pack(pady=10)

audio_path_entry = Entry(root, width=40)
audio_path_entry.pack(pady=10)

browse_button = Button(root, text="Browse", command=browse_file)
browse_button.pack(pady=10)

detect_button = Button(root, text="Detect Emotion", command=detect_emotion_from_audio)
detect_button.pack(pady=20)

result_label= Label(root, text="")
result_label.pack()

root.mainloop()